# Workshop 4: RAG Chatbot with FAISS, LangChain, and Azure OpenAI Function Calling

This notebook walks through building a Retrieval-Augmented Generation (RAG) chatbot using:
- FAISS for vector search
- LangChain for retrieval and conversation orchestration
- Azure OpenAI (Chat + Embeddings) with function/tool calling to extend capabilities

You'll be able to: generate embeddings, build a FAISS index over mock data, retrieve relevant context, chat with the model, and invoke functions like checking device status or creating an IT ticket.

## Prerequisites

- Python 3.9+
- Install dependencies (if running locally):
  - langchain, langchain-openai, langchain-community, faiss-cpu, openai, tiktoken, python-dotenv, jupyter, ipykernel
- Azure OpenAI resource with a Chat model (e.g., gpt-4o-mini) and an Embeddings model (e.g., text-embedding-3-large) deployed.

Environment variables (use a .env file or set in your shell):
- AZURE_OPENAI_API_KEY=...
- AZURE_OPENAI_ENDPOINT=https://<your-azure-openai>.openai.azure.com/
- AZURE_OPENAI_API_VERSION=2024-07-01-preview
- AZURE_OPENAI_CHAT_DEPLOYMENT=gpt-4o-mini (your chat deployment name)
- AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT=text-embedding-3-large (your embeddings deployment name)

In [ ]:
# Setup & imports
import os, json, textwrap
from typing import List, Tuple, Dict, Any

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# LangChain & Vector store
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.chains import ConversationalRetrievalChain

# OpenAI SDK (Azure) for tool/function calling
from openai import AzureOpenAI

# Utility
def require_env(names: List[str]):
    missing = [n for n in names if not os.getenv(n)]
    if missing:
        print("[WARN] Missing env vars ->", missing)
    else:
        print("[OK] Environment variables detected.")

require_env([
    'AZURE_OPENAI_API_KEY',
    'AZURE_OPENAI_ENDPOINT',
    'AZURE_OPENAI_API_VERSION',
    'AZURE_OPENAI_CHAT_DEPLOYMENT',
    'AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'
])

## Mock data: IT Helpdesk FAQ
You can swap this with your team's domain (e.g., HR, Sales).

In [ ]:
mock_docs = [
    {
        'page_content': 'How to reset my password? Visit the password reset page and follow the emailed instructions.',
        'metadata': {'source': 'FAQ - Password Reset'}
    },
    {
        'page_content': 'My computer is slow. Restart it, close unused apps, and run antivirus scans.',
        'metadata': {'source': 'FAQ - Performance Issues'}
    },
    {
        'page_content': 'To connect to VPN, install the client from IT portal and login with your credentials.',
        'metadata': {'source': 'FAQ - VPN Setup'}
    },
    {
        'page_content': 'Printer not working? Ensure it’s powered on, connected, and has ink and paper.',
        'metadata': {'source': 'FAQ - Printer Troubleshooting'}
    },
]
len(mock_docs)

## Build embeddings and FAISS index

In [ ]:
# Configure Azure OpenAI clients
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION', '2024-07-01-preview')
AZURE_OPENAI_CHAT_DEPLOYMENT = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT')
AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT = os.getenv('AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT')

# Embeddings for FAISS
emb_kwargs = dict(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
)
if AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT:
    emb_kwargs['azure_deployment'] = AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT

embeddings = AzureOpenAIEmbeddings(**emb_kwargs)

texts = [d['page_content'] for d in mock_docs]
metas = [d['metadata'] for d in mock_docs]
vectorstore = FAISS.from_texts(texts, embedding=embeddings, metadatas=metas)
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

print('FAISS index built. Vector count:', vectorstore.index.ntotal)

## Set up Conversational Retrieval Chain

In [ ]:
chat = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_deployment=AZURE_OPENAI_CHAT_DEPLOYMENT,
    temperature=0.2,
)

retrieval_chain = ConversationalRetrievalChain.from_llm(
    llm=chat,
    retriever=retriever,
    return_source_documents=True,
)

# History as a list of (user, assistant) tuples
chat_history: List[Tuple[str, str]] = []
print('ConversationalRetrievalChain ready.')

## Define functions (tools) for Azure OpenAI to call
We'll implement two example tools: check_system_status and create_it_ticket.

In [ ]:
# Mock functions
def check_system_status(device_id: str) -> str:
    status_map = {
        'printer01': 'Online and functioning normally.',
        'router23': 'Offline - requires restart.',
        'server07': 'Online but high CPU usage.',
    }
    return status_map.get(device_id, 'Device not found.')

def create_it_ticket(issue: str, urgency: str = 'medium') -> str:
    import random
    ticket_id = f'TKT-{random.randint(1000,9999)}'
    return f'Ticket {ticket_id} created for issue: 
 (urgency={urgency}).'

# Tool schema for Azure OpenAI (Chat Completions)
tools_schema = [
    {
        'type': 'function',
        'function': {
            'name': 'check_system_status',
            'description': 'Checks device status by device ID',
            'parameters': {
                'type': 'object',
                'properties': {
                    'device_id': {'type': 'string', 'description': 'Device unique identifier'}
                },
                'required': ['device_id']
            }
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'create_it_ticket',
            'description': 'Creates an IT support ticket for a described issue',
            'parameters': {
                'type': 'object',
                'properties': {
                    'issue': {'type': 'string', 'description': 'Users issue description'},
                    'urgency': {'type': 'string', 'enum': ['low','medium','high']}
                },
                'required': ['issue']
            }
        }
    }
]

# Azure OpenAI client
aoai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

SYSTEM_PROMPT = (
    'You are an IT helpdesk assistant. Use the provided knowledge and call tools when helpful. '
)

def call_tools_if_requested(message: Dict[str, Any]) -> List[Dict[str, str]]:
    """Handle tool calls by executing local Python functions and returning tool outputs as messages.
    Returns a list of new messages (assistant tool calls + tool outputs) to append.
    """
    followups = []
    tool_calls = message.get('tool_calls') or []
    for tool_call in tool_calls:
        func = tool_call['function']['name']
        args_json = tool_call['function'].get('arguments') or '{}'
        try:
            args = json.loads(args_json) if isinstance(args_json, str) else args_json
        except Exception:
            args = {}
        # Execute corresponding function
        if func == 'check_system_status':
            out = check_system_status(**args)
        elif func == 'create_it_ticket':
            out = create_it_ticket(**args)
        else:
            out = f'Unknown tool: {func}'
        # Represent the tool result as a tool message
        followups.append({
            'role': 'tool',
            'tool_call_id': tool_call.get('id', ''),
            'name': func,
            'content': out,
        })
    return followups

def chat_with_rag_and_tools(query: str, chat_history: List[Tuple[str, str]]):
    # 1) Retrieve knowledge via LangChain
    rag = retrieval_chain({'question': query, 'chat_history': chat_history})
    sources = rag.get('source_documents', [])
    knowledge = '

'.join([f"- {d.metadata.get('source')}: {d.page_content}" for d in sources])

    # 2) Build messages for Azure OpenAI tools
    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'system', 'content': f'Relevant knowledge:
{knowledge}' if knowledge else 'No knowledge retrieved.'}
    ]
    for q, a in chat_history:
        messages.append({'role': 'user', 'content': q})
        messages.append({'role': 'assistant', 'content': a})
    messages.append({'role': 'user', 'content': query})

    # 3) First model call (may request tools)
    first = aoai_client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_DEPLOYMENT,
        messages=messages,
        tools=tools_schema,
        tool_choice='auto'
    )
    msg = first.choices[0].message

    # 4) If tools requested, execute and send follow-up
    if getattr(msg, 'tool_calls', None):
        # Append assistant tool-calling message
        messages.append({'role': 'assistant', 'content': msg.content or '', 'tool_calls': [tc.model_dump() for tc in msg.tool_calls]})
        # Call local tools
        for tc in msg.tool_calls:
            func = tc.function.name
            args = tc.function.arguments
            try:
                parsed = json.loads(args) if isinstance(args, str) else args
            except Exception:
                parsed = {}
            if func == 'check_system_status':
                out = check_system_status(**parsed)
            elif func == 'create_it_ticket':
                out = create_it_ticket(**parsed)
            else:
                out = f'Unknown tool: {func}'
            messages.append({
                'role': 'tool',
                'tool_call_id': tc.id,
                'name': func,
                'content': out,
            })
        # 5) Second model call to get final answer
        second = aoai_client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT,
            messages=messages
        )
        final_msg = second.choices[0].message
        answer = final_msg.content
    else:
        answer = msg.content

    chat_history.append((query, answer))
    return {
        'answer': answer,
        'sources': [
            {'source': d.metadata.get('source'), 'content': d.page_content} for d in sources
        ]
    }

print('Tools and Azure OpenAI client configured.')

## Try it: single query

In [ ]:
query = 'Printer not working. What should I do? Also check printer01 status.'
result = chat_with_rag_and_tools(query, chat_history)
print('Answer:
', textwrap.fill(result['answer'], 100))
print('
Sources:')
for s in result['sources']:
    print('-', s['source'])

## Multi-turn demo

In [ ]:
turns = [
    'How do I reset my password?',
    'My computer is slow, what should I try?',
    'Create a ticket for slow performance with high urgency.',
]
for t in turns:
    r = chat_with_rag_and_tools(t, chat_history)
    print('
User:', t)
    print('Assistant:', textwrap.fill(r['answer'], 100))
print('
Done. Conversation length:', len(chat_history))